On the curve $x^4 + y^4 = 1$, use the inner product
$$
\langle f, g \rangle = \int_{-1}^{1} \left[ f(x,y)g(x,y) + f(x,-y)g(x,-y)\right]\mathrm{d}x, \qquad y = \sqrt[4]{1-x^4}.
$$

The orthonormal polynomials satisfy
$$
xQ_n = \left( B_{n-1}^x \right)^{\intercal} Q_{n-1} + A_n^x Q_n + B_{n}^xQ_{n+1}, \qquad Q_n = \left(\begin{array}{c} p_n(x,y) \\
q_n(x,y) \\
r_n(x,y) \\
s_n(x,y)
\end{array}  \right).
$$
Following the Gram-Schmidt procedure and using properties of the inner product reveal that $A_n^x$ and $A_n^y$ are zero and the entries of the $B$-matrices are:
$$ B_n^x = \left( 
\begin{array}{c c c c}
\| u_1 \| &             &   &     \\
          & \| u_2 \|   &   &      \\
\langle u_3,p_{n+1} \rangle &   & \| v_3 \| & \\
 & \langle u_4, q_{n+1} \rangle &  & \|v_4 \|
\end{array}
\right), \qquad
B_n^y = \left( 
\begin{array}{c c c c}
 & \langle u_5, q_{n+1}\rangle &    &  \langle u_5, s_{n+1}\rangle    \\
\langle u_6, p_{n+1}\rangle &  & \langle u_6, r_{n+1}\rangle   &      \\
 & \langle u_7, q_{n+1}\rangle &    &  \langle u_7, s_{n+1}\rangle  \\
\langle u_8, p_{n+1}\rangle &  & \langle u_8, r_{n+1}\rangle   &      \\
\end{array}
\right),
$$
where
$$
\left(
\begin{array}{l}
u_1   \\
u_2  \\
u_3   \\
u_4 
\end{array}
\right) = xQ_n - \left(B_{n-1}^x\right)^{\intercal}Q_{n-1}  = B_n^x Q_{n+1}= \left(
\begin{array}{l}
xp_n - b_{1,1}^{n-1,x} p_{n-1} - b_{3,1}^{n-1,x}r_{n-1}   \\
xq_n - b_{2,2}^{n-1,x} q_{n-1} - b_{4,2}^{n-1,x} s_{n-1}   \\
xr_n - b_{3,3}^{n-1,x} r_{n-1}                    \\
xs_n - b_{4,4}^{n-1,x} s_{n-1}                 
\end{array}
\right),
$$
$$
\left(
\begin{array}{l}
u_5  \\
u_6   \\
u_7  \\
u_8 
\end{array}
\right) = yQ_{n} - \left(B_{n-1}^y\right)^{\intercal}Q_{n-1}  = B_n^y Q_{n+1}= \left(
\begin{array}{l}
yp_n - b_{2,1}^{n-1,y} q_{n-1} - b_{4,1}^{n-1,y} s_{n-1}     \\
yq_n - b_{1,2}^{n-1,y} p_{n-1} - b_{3,2}^{n-1,y}r_{n-1}   \\
yr_n - b_{2,3}^{n-1,y} q_{n-1} - b_{4,3}^{n-1,y} s_{n-1}                  \\
ys_n - b_{1,4}^{n-1,y} p_{n-1} - b_{3,4}^{n-1,y}r_{n-1}                
\end{array}
\right).
$$

Here are approximations to $B_{1000}^x$ and $B_{1000}^y$, obtained with Gauss-Legendre quadrature in 100,000 points:

In [1]:
using Optim, LinearAlgebra

In [4]:
bx=zeros(4,4)
bx[1,1]=0.5000754998732936;bx[2,2]=0.5000000000002628;bx[3,3]=0.49995057513142593;bx[4,4]=0.9975104173927817
bx[3,1]=-2.05454176225822e-5; bx[4,2]=-0.049832678693653126
bx

4×4 Array{Float64,2}:
  0.500075     0.0        0.0       0.0    
  0.0          0.5        0.0       0.0    
 -2.05454e-5   0.0        0.499951  0.0    
  0.0         -0.0498327  0.0       0.99751

In [5]:
by=zeros(4,4)
by[1,2]=0.6216126011671426;by[1,4]=0.015966548139465582;by[2,1]=-0.435531582746482;by[2,3]=0.09748050098342892
by[3,2]=0.022262136962950867;by[3,4]=0.023518966900343975;by[4,1]=-0.04607472705992894;by[4,3]=0.005005078678931155
by

4×4 Array{Float64,2}:
  0.0        0.621613   0.0         0.0159665
 -0.435532   0.0        0.0974805   0.0      
  0.0        0.0222621  0.0         0.023519 
 -0.0460747  0.0        0.00500508  0.0      

Try to find $B_{\infty}^x$ and $B_{\infty}^y$ by nonlinear optimisation using $B_{1000}^x$ and $B_{1000}^y$ as first guesses:

In [8]:
function unroll(XY_in)
    XY = reshape(XY_in,8,4)
    Bx = XY[1:4,:]; By = XY[5:8,:];
    Bx, By
end
function eqns(XY_in)
    Bx, By = unroll(XY_in)
    symsumx = Bx'*Bx + Bx*Bx';
    symsumy = By'*By + By*By';
    # algebraic equation
    norm(Bx^4 + By^4)^2 + 
    norm(symsumx*Bx^2 + Bx^2*symsumx + symsumy*By^2 + By^2*symsumy)^2 +
    norm((Bx')^2*Bx^2 + symsumx^2 + Bx^2*(Bx')^2 + (By')^2*By^2 + symsumy^2 + By^2*(By')^2 - I )^2 + 
    # commute
    norm(Bx*By - By*Bx)^2 +  norm(Bx'*By + Bx*By' - (By'*Bx + By*Bx'))^2 +
    # z = 1
    norm(Bx' + Bx - I)^2 + norm(By' + By)^2 +
    # z = i
    norm(-Bx' + Bx)^2 + norm(-By' + By - [0 0 0 -1;0 0 1 0;0 -1 0 0;1 0 0 0])^2
end;

In [21]:
guess = [bx;by]
# result = optimize(eqns, Array(reshape(guess,32,1))) # fails
# result = optimize(eqns, Array(reshape(guess,32,1)),Newton()) # fails
# BFGS, ConjugateGradient and NGMRES converge to the same answer
result = optimize(eqns, Array(reshape(guess,32,1)),BFGS())
# result = optimize(eqns, Array(reshape(guess,32,1)),ConjugateGradient())
# result = optimize(eqns, Array(reshape(guess,32,1)),NGMRES())

 * Status: success

 * Candidate solution
    Minimizer: [5.29e-01, 0.00e+00, 9.65e-12,  ...]
    Minimum:   1.395549e-01

 * Found with
    Algorithm:     BFGS
    Initial Point: [5.00e-01, 0.00e+00, -2.05e-05,  ...]

 * Convergence measures
    |x - x'|               = 4.49e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.49e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.78e-17 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.99e-16 ≰ 0.0e+00
    |g(x)|                 = 3.60e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    39
    f(x) calls:    115
    ∇f(x) calls:   115


In [20]:
Bx, By = unroll(Optim.minimizer(result))
Bx

4×4 Array{Float64,2}:
  0.529184     0.0          -6.29762e-11   0.0        
  0.0          0.529184      0.0          -8.25664e-11
 -1.91225e-11  0.0           0.529184      0.0        
  0.0          1.01559e-11   0.0           0.529184   

In [18]:
By

4×4 Array{Float64,2}:
 0.0           7.6762e-11  0.0          -0.529184   
 4.70015e-11   0.0         0.529184      0.0        
 0.0          -0.529184    0.0           9.11869e-11
 0.529184      0.0         1.48927e-10   0.0        

It is clear from the output of the optimiser that the minimum found is not a good one. We can confirm this:

In [23]:
X=z->Bx'/z+Bx*z
Y=z->By'/z+By*z;
theta = pi/4;
z = exp(im*theta)
commute = norm(X(z)*Y(z) - Y(z)*X(z))

2.567352479236712e-10

In [24]:
algeq = norm(X(z)^4 + Y(z)^4 -I)

0.7452830368770653